# Battery Storage Module

Models lithium-ion battery energy storage system (BESS) for managing temporal mismatch between generation and demand. Includes SOC dynamics and exergy analysis.

## Energy Analysis

**State of charge dynamics:**
$$SOC(t + \Delta t) = SOC(t) + \frac{\eta_{ch} P_{ch}(t) \Delta t}{E_{nom}} - \frac{P_{dch}(t) \Delta t}{\eta_{dch} E_{nom}}$$

**Round-trip efficiency:**
$$\eta_{rt} = \eta_{ch} \cdot \eta_{dch}$$

Symmetric assumption: $\eta_{ch} = \eta_{dch} = \sqrt{\eta_{rt}}$

NREL benchmark: $\eta_{rt} = 0.86$ (86%)

**Operational constraints:**
$$SOC_{min} \leq SOC(t) \leq SOC_{max}$$
$$0 \leq P_{ch}(t) \leq P_{max}, \quad 0 \leq P_{dch}(t) \leq P_{max}$$
$$P_{ch}(t) \cdot P_{dch}(t) = 0 \quad \text{(no simultaneous charge/discharge)}$$

## Exergy Analysis

**Exergy stored:**
$$Ex_{sto}(t) = SOC(t) \cdot E_{nom}$$

**Exergy destruction (charging):**
$$\dot{Ex}_{d,ch}(t) = P_{ch}(t) (1 - \eta_{ch})$$

**Exergy destruction (discharging):**
$$\dot{Ex}_{d,dch}(t) = \frac{P_{dch}(t)}{\eta_{dch}} - P_{dch}(t) = P_{dch}(t)\left(\frac{1}{\eta_{dch}} - 1\right)$$

**Exergy efficiency (round-trip):**
$$\Psi_{bat} = \eta_{rt}$$

Note: For electrical storage, exergy efficiency equals energy efficiency since electricity is pure exergy.

## Parameters

In [1]:
# Battery Storage Default Parameters
const BAT_DEFAULTS = (
    η_rt = 0.86,            # Round-trip efficiency [-] (NREL 2021)
    SOC_min = 0.1,          # Minimum SOC [-]
    SOC_max = 0.9,          # Maximum SOC [-]
    SOC_init = 0.5,         # Initial SOC [-]
    C_rate_max = 1.0        # Maximum C-rate [1/h]
)

(η_rt = 0.86, SOC_min = 0.1, SOC_max = 0.9, SOC_init = 0.5, C_rate_max = 1.0)

## Core Functions

In [2]:
"""
    BatteryStorage

Struct holding battery storage model parameters.
"""
struct BatteryStorage
    E_nom::Float64          # Nominal energy capacity [Wh]
    P_max::Float64          # Maximum charge/discharge power [W]
    η_rt::Float64           # Round-trip efficiency [-]
    η_ch::Float64           # Charging efficiency [-]
    η_dch::Float64          # Discharging efficiency [-]
    SOC_min::Float64        # Minimum SOC [-]
    SOC_max::Float64        # Maximum SOC [-]
    
    function BatteryStorage(E_nom::Float64, P_max::Float64;
            η_rt::Float64 = 0.86,
            SOC_min::Float64 = 0.1,
            SOC_max::Float64 = 0.9)
        # Symmetric efficiencies
        η_ch = sqrt(η_rt)
        η_dch = sqrt(η_rt)
        new(E_nom, P_max, η_rt, η_ch, η_dch, SOC_min, SOC_max)
    end
end

# Constructor from capacity in kWh
function BatteryStorage(; E_kWh::Float64, P_kW::Float64, kwargs...)
    BatteryStorage(E_kWh * 1000.0, P_kW * 1000.0; kwargs...)
end

BatteryStorage

In [3]:
"""
    usable_capacity(bat::BatteryStorage) -> Float64

Calculate usable energy capacity [Wh] considering SOC limits.
"""
function usable_capacity(bat::BatteryStorage)
    return bat.E_nom * (bat.SOC_max - bat.SOC_min)
end

usable_capacity

In [4]:
"""
    update_soc(bat::BatteryStorage, SOC::Float64, P_ch::Float64, P_dch::Float64, Δt::Float64) -> Float64

Update state of charge after time step Δt [hours].
P_ch, P_dch in [W], returns new SOC [-].
"""
function update_soc(bat::BatteryStorage, SOC::Float64, P_ch::Float64, P_dch::Float64, Δt::Float64)
    # Energy changes [Wh]
    E_in = bat.η_ch * P_ch * Δt      # Energy stored from charging
    E_out = P_dch * Δt / bat.η_dch   # Energy withdrawn (before losses)
    
    # Update SOC
    SOC_new = SOC + E_in / bat.E_nom - E_out / bat.E_nom
    
    # Clamp to limits
    return clamp(SOC_new, bat.SOC_min, bat.SOC_max)
end

update_soc

In [5]:
"""
    max_charge_power(bat::BatteryStorage, SOC::Float64, Δt::Float64) -> Float64

Maximum charging power [W] respecting SOC_max and P_max.
"""
function max_charge_power(bat::BatteryStorage, SOC::Float64, Δt::Float64)
    # Energy headroom to SOC_max
    E_headroom = (bat.SOC_max - SOC) * bat.E_nom  # [Wh]
    # Power limited by headroom (accounting for efficiency)
    P_soc_limit = E_headroom / (bat.η_ch * Δt)
    return min(bat.P_max, P_soc_limit)
end

max_charge_power

In [6]:
"""
    max_discharge_power(bat::BatteryStorage, SOC::Float64, Δt::Float64) -> Float64

Maximum discharging power [W] respecting SOC_min and P_max.
"""
function max_discharge_power(bat::BatteryStorage, SOC::Float64, Δt::Float64)
    # Energy available above SOC_min
    E_available = (SOC - bat.SOC_min) * bat.E_nom  # [Wh]
    # Power limited by available energy (accounting for efficiency)
    P_soc_limit = E_available * bat.η_dch / Δt
    return min(bat.P_max, P_soc_limit)
end

max_discharge_power

In [7]:
"""
    exergy_destruction_charge(bat::BatteryStorage, P_ch::Float64) -> Float64

Exergy destruction rate [W] during charging.
"""
function exergy_destruction_charge(bat::BatteryStorage, P_ch::Float64)
    return P_ch * (1.0 - bat.η_ch)
end

exergy_destruction_charge

In [8]:
"""
    exergy_destruction_discharge(bat::BatteryStorage, P_dch::Float64) -> Float64

Exergy destruction rate [W] during discharging.
"""
function exergy_destruction_discharge(bat::BatteryStorage, P_dch::Float64)
    # Exergy from battery = P_dch / η_dch, useful output = P_dch
    return P_dch * (1.0 / bat.η_dch - 1.0)
end

exergy_destruction_discharge

In [9]:
"""
    analyze_battery(bat::BatteryStorage, SOC::Float64, P_ch::Float64, P_dch::Float64, Δt::Float64) -> NamedTuple

Complete energy and exergy analysis for a time step.
P_ch, P_dch in [W], Δt in [hours].
"""
function analyze_battery(bat::BatteryStorage, SOC::Float64, P_ch::Float64, P_dch::Float64, Δt::Float64)
    # Validate inputs
    @assert P_ch >= 0 && P_dch >= 0 "Powers must be non-negative"
    @assert P_ch == 0 || P_dch == 0 "Cannot charge and discharge simultaneously"
    
    # Energy analysis
    SOC_new = update_soc(bat, SOC, P_ch, P_dch, Δt)
    E_stored = SOC * bat.E_nom
    E_stored_new = SOC_new * bat.E_nom
    ΔE = E_stored_new - E_stored  # Change in stored energy [Wh]
    
    # Exergy analysis
    Ex_stored = E_stored          # Electrical exergy = energy
    Ex_stored_new = E_stored_new
    Ex_d_ch = exergy_destruction_charge(bat, P_ch) * Δt    # [Wh]
    Ex_d_dch = exergy_destruction_discharge(bat, P_dch) * Δt  # [Wh]
    Ex_d_total = Ex_d_ch + Ex_d_dch
    
    # Instantaneous exergy efficiency
    if P_ch > 0
        Ψ_inst = bat.η_ch
    elseif P_dch > 0
        Ψ_inst = bat.η_dch
    else
        Ψ_inst = 1.0  # Idle
    end
    
    return (
        SOC = SOC,                # Initial SOC [-]
        SOC_new = SOC_new,        # Final SOC [-]
        P_ch = P_ch,              # Charge power [W]
        P_dch = P_dch,            # Discharge power [W]
        E_stored = E_stored,      # Initial stored energy [Wh]
        E_stored_new = E_stored_new,  # Final stored energy [Wh]
        ΔE = ΔE,                  # Energy change [Wh]
        Ex_d = Ex_d_total,        # Total exergy destruction [Wh]
        Ψ_inst = Ψ_inst,          # Instantaneous exergy efficiency [-]
        Ψ_rt = bat.η_rt           # Round-trip exergy efficiency [-]
    )
end

analyze_battery

## Tests

In [10]:
using Test

function run_battery_tests()
    println("Running Battery Storage Tests...")
    
    # Create test battery: 10 kWh, 5 kW
    bat = BatteryStorage(E_kWh=10.0, P_kW=5.0, η_rt=0.86)
    
    # Test 1: Parameter calculation
    @testset "Parameters" begin
        @test bat.E_nom == 10000.0  # Wh
        @test bat.P_max == 5000.0   # W
        @test bat.η_ch ≈ sqrt(0.86) atol=1e-6
        @test bat.η_dch ≈ sqrt(0.86) atol=1e-6
        @test bat.η_ch * bat.η_dch ≈ bat.η_rt atol=1e-6
    end
    
    # Test 2: Usable capacity
    @testset "Usable Capacity" begin
        E_usable = usable_capacity(bat)
        @test E_usable ≈ 10000.0 * (0.9 - 0.1) atol=1e-6
    end
    
    # Test 3: SOC update - charging
    @testset "SOC Charging" begin
        SOC_init = 0.5
        P_ch = 2000.0  # 2 kW
        Δt = 1.0       # 1 hour
        SOC_new = update_soc(bat, SOC_init, P_ch, 0.0, Δt)
        # Expected: SOC + η_ch * P_ch * Δt / E_nom
        expected = SOC_init + bat.η_ch * P_ch * Δt / bat.E_nom
        @test SOC_new ≈ expected atol=1e-6
        @test SOC_new > SOC_init
    end
    
    # Test 4: SOC update - discharging
    @testset "SOC Discharging" begin
        SOC_init = 0.5
        P_dch = 2000.0  # 2 kW
        Δt = 1.0
        SOC_new = update_soc(bat, SOC_init, 0.0, P_dch, Δt)
        # Expected: SOC - P_dch * Δt / (η_dch * E_nom)
        expected = SOC_init - P_dch * Δt / (bat.η_dch * bat.E_nom)
        @test SOC_new ≈ expected atol=1e-6
        @test SOC_new < SOC_init
    end
    
    # Test 5: SOC limits enforced
    @testset "SOC Limits" begin
        # Try to overcharge
        SOC_new = update_soc(bat, 0.85, 5000.0, 0.0, 1.0)
        @test SOC_new <= bat.SOC_max
        # Try to over-discharge
        SOC_new = update_soc(bat, 0.15, 0.0, 5000.0, 1.0)
        @test SOC_new >= bat.SOC_min
    end
    
    # Test 6: Round-trip efficiency
    @testset "Round-Trip Efficiency" begin
        # Charge then discharge same energy
        SOC_init = 0.5
        P = 2000.0
        Δt = 1.0
        
        # Charge
        SOC_charged = update_soc(bat, SOC_init, P, 0.0, Δt)
        E_in = P * Δt  # Energy from grid
        
        # Discharge back to original SOC
        E_stored_added = (SOC_charged - SOC_init) * bat.E_nom
        E_out = E_stored_added * bat.η_dch  # Energy to grid
        
        # Round-trip efficiency
        η_measured = E_out / E_in
        @test η_measured ≈ bat.η_rt atol=0.01
    end
    
    # Test 7: Exergy destruction
    @testset "Exergy Destruction" begin
        P_ch = 3000.0
        Ex_d = exergy_destruction_charge(bat, P_ch)
        @test Ex_d ≈ P_ch * (1 - bat.η_ch) atol=1e-6
        @test Ex_d > 0
        
        P_dch = 3000.0
        Ex_d = exergy_destruction_discharge(bat, P_dch)
        @test Ex_d ≈ P_dch * (1/bat.η_dch - 1) atol=1e-6
        @test Ex_d > 0
    end
    
    # Test 8: Full analysis
    @testset "Full Analysis" begin
        result = analyze_battery(bat, 0.5, 2000.0, 0.0, 1.0)
        @test result.SOC_new > result.SOC
        @test result.Ex_d > 0
        @test result.Ψ_inst ≈ bat.η_ch atol=1e-6
    end
    
    # Test 9: No simultaneous charge/discharge
    @testset "Complementarity" begin
        @test_throws AssertionError analyze_battery(bat, 0.5, 1000.0, 1000.0, 1.0)
    end
    
    println("All tests passed!")
end

run_battery_tests()

Running Battery Storage Tests...
Test Summary: | Pass  Total  Time
Parameters    |    5      5  0.0s
Test Summary:   | Pass  Total  Time
Usable Capacity |    1      1  0.0s
Test Summary: | Pass  Total  Time
SOC Charging  |    2      2  0.0s
Test Summary:   | Pass  Total  Time
SOC Discharging |    2      2  0.0s
Test Summary: | Pass  Total  Time
SOC Limits    |    2      2  0.0s
Test Summary:         | Pass  Total  Time
Round-Trip Efficiency |    1      1  0.0s
Test Summary:      | Pass  Total  Time
Exergy Destruction |    4      4  0.0s
Test Summary: | Pass  Total  Time
Full Analysis |    3      3  0.0s
Test Summary:   | Pass  Total  Time
Complementarity |    1      1  0.0s
All tests passed!


## Example Usage

In [11]:
# Example: 10 kWh battery, 5 kW max power
bat = BatteryStorage(E_kWh=10.0, P_kW=5.0, η_rt=0.86)

println("Battery Storage Specifications")
println("="^40)
println("Nominal capacity:   $(bat.E_nom/1000) kWh")
println("Usable capacity:    $(usable_capacity(bat)/1000) kWh")
println("Max power:          $(bat.P_max/1000) kW")
println("Round-trip eff:     $(round(bat.η_rt*100, digits=1))%")
println("Charge eff:         $(round(bat.η_ch*100, digits=1))%")
println("Discharge eff:      $(round(bat.η_dch*100, digits=1))%")
println("SOC range:          $(bat.SOC_min*100)% - $(bat.SOC_max*100)%")

Battery Storage Specifications
Nominal capacity:   10.0 kWh
Usable capacity:    8.0 kWh
Max power:          5.0 kW
Round-trip eff:     86.0%
Charge eff:         92.7%
Discharge eff:      92.7%
SOC range:          10.0% - 90.0%


In [12]:
# Simulate charge-discharge cycle (wrapped in let block for NBInclude compatibility)
let
    println("\nCharge-Discharge Cycle Simulation")
    println("="^50)

    SOC = 0.2
    Δt = 1.0  # 1 hour steps

    println("Hour  Action      Power[kW]  SOC[%]   Ex_d[Wh]")
    println("─"^50)

    # Charge for 3 hours at 3 kW
    for t in 1:3
        result = analyze_battery(bat, SOC, 3000.0, 0.0, Δt)
        println("$(lpad(t, 4))  Charge      $(lpad(3.0, 6))     $(lpad(round(result.SOC_new*100, digits=1), 5))    $(lpad(round(result.Ex_d, digits=1), 7))")
        SOC = result.SOC_new
    end

    # Discharge for 3 hours at 2.5 kW
    for t in 4:6
        result = analyze_battery(bat, SOC, 0.0, 2500.0, Δt)
        println("$(lpad(t, 4))  Discharge   $(lpad(2.5, 6))     $(lpad(round(result.SOC_new*100, digits=1), 5))    $(lpad(round(result.Ex_d, digits=1), 7))")
        SOC = result.SOC_new
    end
end


Charge-Discharge Cycle Simulation
Hour  Action      Power[kW]  SOC[%]   Ex_d[Wh]
──────────────────────────────────────────────────
   1  Charge         3.0      47.8      217.9
   2  Charge         3.0      75.6      217.9
   3  Charge         3.0      90.0      217.9
   4  Discharge      2.5      63.0      195.8
   5  Discharge      2.5      36.1      195.8
   6  Discharge      2.5      10.0      195.8


## Export

Functions available for import:
- `BatteryStorage` - Model struct (accepts E_nom/P_max or E_kWh/P_kW)
- `usable_capacity` - Usable energy considering SOC limits
- `update_soc` - Update SOC for a time step
- `max_charge_power` - Maximum charging power at given SOC
- `max_discharge_power` - Maximum discharging power at given SOC
- `exergy_destruction_charge` - Exergy destruction during charging
- `exergy_destruction_discharge` - Exergy destruction during discharging
- `analyze_battery` - Complete analysis for a time step